# 02_Preprocessing

Before we engineer our recommender system, we must preprocess our data for a robust system. For our content-based recommender, we need to use standard nlp practices such as tokenize and lemmatize to turn text data into numerical data. For the collaborative filtering recommender system, it is important to remove item bias by calculating the average score and of the given reviews for that movie as well as calculating the standard deviation. By subtracting the mean review score from the review score and dividing by the standard deviation, we are essentially standard scaling and the transformed values now carry more meaning. The calculations were performed server-side postgressql for computational efficiency. 

In [1]:
!pip install nltk

In [2]:
import nltk
nltk.download('wordnet')

import re
from nltk.tokenize import RegexpTokenizer

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

import pandas as pd
import numpy as np

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Content-based recommender preprocessing

In [3]:
merged = pd.read_csv('../assets/merged.csv', lineterminator='\n')

In [4]:
merged.isnull().sum()

Unnamed: 0              0
movie_id                0
tmdb_id                 0
release_date           12
overview              210
title_y                 0
genres                  0
overview_tokenized      0
overview_features     213
dtype: int64

In [5]:
merged.shape

(26742, 9)

In [6]:
merged.head()

,Unnamed: 0,movie_id,tmdb_id,release_date,overview,title_y,genres,overview_tokenized,overview_features
0,0,1,862,1995-10-30,"Led by Woody, Andy's toys live happily in his ...",Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"['Led', 'by', 'Woody', 'Andy', 's', 'toys', 'l...",Led by Woody Andy s toy live happily in his ro...
1,1,2,8844,1995-12-15,When siblings Judy and Peter discover an encha...,Jumanji (1995),Adventure|Children|Fantasy,"['When', 'siblings', 'Judy', 'and', 'Peter', '...",When sibling Judy and Peter discover an enchan...
2,2,3,15602,1995-12-22,A family wedding reignites the ancient feud be...,Grumpier Old Men (1995),Comedy|Romance,"['A', 'family', 'wedding', 'reignites', 'the',...",A family wedding reignites the ancient feud be...
3,3,129,110972,1996-02-09,Pie in the Sky is a 1996 American romantic com...,Pie in the Sky (1996),Comedy|Romance,"['Pie', 'in', 'the', 'Sky', 'is', 'a', '1996',...",Pie in the Sky is a 1996 American romantic com...
4,4,654,278978,1996-02-29,NaN,And Nobody Weeps for Me (Und keiner weint mir ...,Drama|Romance,[],NaN


In [7]:
tokenizer = RegexpTokenizer(r'\w+')

In [8]:
merged['genres_split'] = merged['genres'].str.split('|')

In [9]:
merged['genres_features'] = merged['genres_split'].map(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x]))


In [10]:
merged.isnull().sum()

Unnamed: 0              0
movie_id                0
tmdb_id                 0
release_date           12
overview              210
title_y                 0
genres                  0
overview_tokenized      0
overview_features     213
genres_split            0
genres_features         0
dtype: int64

We have null values in our overview. In order to run our preprocessing, I will fill them with empty strings.

In [11]:
merged[merged.overview.isnull()].head()

,Unnamed: 0,movie_id,tmdb_id,release_date,overview,title_y,genres,overview_tokenized,overview_features,genres_split,genres_features
4,4,654,278978,1996-02-29,NaN,And Nobody Weeps for Me (Und keiner weint mir ...,Drama|Romance,[],NaN,"[Drama, Romance]",Drama Romance
10,10,33,78802,1996-09-18,NaN,Wings of Courage (1995),Adventure|Romance|IMAX,[],NaN,"[Adventure, Romance, IMAX]",Adventure Romance IMAX
71,71,4926,58886,2000-04-12,NaN,Everybody's Famous! (Iedereen beroemd!) (2000),Comedy|Drama|Musical,[],NaN,"[Comedy, Drama, Musical]",Comedy Drama Musical
98,98,42636,68646,2002-02-28,NaN,Superproduction (Superprodukcja) (2003),Comedy,[],NaN,[Comedy],Comedy
658,658,643,287305,1996-03-21,NaN,Peanuts - Die Bank zahlt alles (1996),Comedy,[],NaN,[Comedy],Comedy


In [12]:
merged['overview'].isnull().sum()

210

In [13]:
len(merged[merged['overview'] == 'No overview found.'])

35

Total of 245 (=210 + 35) movies do not have an overview and hence will not be included in the content recommender.

In [14]:
merged['overview'].fillna('', inplace = True)
merged['overview'].replace('No overview found.', '')
merged['overview_features'].fillna('', inplace = True)

In [15]:
merged.isnull().sum()

Unnamed: 0             0
movie_id               0
tmdb_id                0
release_date          12
overview               0
title_y                0
genres                 0
overview_tokenized     0
overview_features      0
genres_split           0
genres_features        0
dtype: int64

We do not care about if release_date is null or not; it will be left as is.

Join the genres_features and overview_features to create an aggregate field of features that describe the movie

In [16]:
merged['all_features'] = merged['genres_features'] + ' ' + merged['overview_features']

In [17]:
merged.head()

,Unnamed: 0,movie_id,tmdb_id,release_date,overview,title_y,genres,overview_tokenized,overview_features,genres_split,genres_features,all_features
0,0,1,862,1995-10-30,"Led by Woody, Andy's toys live happily in his ...",Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,"['Led', 'by', 'Woody', 'Andy', 's', 'toys', 'l...",Led by Woody Andy s toy live happily in his ro...,"[Adventure, Animation, Children, Comedy, Fantasy]",Adventure Animation Children Comedy Fantasy,Adventure Animation Children Comedy Fantasy Le...
1,1,2,8844,1995-12-15,When siblings Judy and Peter discover an encha...,Jumanji (1995),Adventure|Children|Fantasy,"['When', 'siblings', 'Judy', 'and', 'Peter', '...",When sibling Judy and Peter discover an enchan...,"[Adventure, Children, Fantasy]",Adventure Children Fantasy,Adventure Children Fantasy When sibling Judy a...
2,2,3,15602,1995-12-22,A family wedding reignites the ancient feud be...,Grumpier Old Men (1995),Comedy|Romance,"['A', 'family', 'wedding', 'reignites', 'the',...",A family wedding reignites the ancient feud be...,"[Comedy, Romance]",Comedy Romance,Comedy Romance A family wedding reignites the ...
3,3,129,110972,1996-02-09,Pie in the Sky is a 1996 American romantic com...,Pie in the Sky (1996),Comedy|Romance,"['Pie', 'in', 'the', 'Sky', 'is', 'a', '1996',...",Pie in the Sky is a 1996 American romantic com...,"[Comedy, Romance]",Comedy Romance,Comedy Romance Pie in the Sky is a 1996 Americ...
4,4,654,278978,1996-02-29,,And Nobody Weeps for Me (Und keiner weint mir ...,Drama|Romance,[],,"[Drama, Romance]",Drama Romance,Drama Romance


In [18]:
merged.drop('Unnamed: 0', axis = 1, inplace = True)

In [19]:
merged.rename(columns = {'title_y': 'title'}, inplace = True)

In [21]:
merged.to_csv('../assets/merged_processed.csv')

### Collaborative Filtering Recommender Preprocessing

Linking up the serial movie_id with the title from original movie database because the cosine_distance_df is currently serial_movie_ids

```
CREATE TABLE serial_movie_id_title(
  s_movie_id INT,
  title TEXT
);
```

I am removing item bias

```
INSERT INTO serial_movie_id_title(
  SELECT a.s_movie_id, b.title
  FROM serial_movie_id AS a
  INNER JOIN movie AS b
  on a.movie_id = b.movie_id
);
 ```

Create a table with scaler that selects the movie_id and the average of ratings and standard deviation of movies that have more than 9 reviews:
```
CREATE TABLE scalers AS(
SELECT movie_id, avg(rating), stddev(rating)
FROM ratings GROUP BY movie_id
HAVING COUNT(*) > 9
);
```

Scale the data using the calculated fields from above:
```
CREATE TABLE scaled AS(
SELECT a.movie_id, a.user_id,
(a.rating - b.avg)/b.stddev AS scaled_rating
FROM ratings AS a
INNER JOIN scalers AS b
ON a.movie_id = b.movie_id
);
```

The preprocessed dataframe and scaled table will now be utilized in **03_Recommender_Engineering.ipynb** 